In [1]:
import glob
import zipfile
import json
import pandas as pd
import pickle
from pprint import pprint

In [3]:
data_path = "/opt/datasets/microsoft_kdd/"

In [ ]:
paper_file = data_path + "Papers.txt"

paper_dict = {}
title_dict = {}

with open(paper_file, "r") as f:
    for line in f:
        paper_elems = line.split(" ")
        pid = paper_elems[0]
        
        normalized_title = ''.join(filter(str.isalpha, paper_elems[1]))
        
        paper_dict[pid] = paper_elems[1:]
        title_dict[normalized_title] = pid